### 기본 Library 선언 및 Tensorflow 버전 확인

In [13]:
import tensorflow as tf
import numpy as np

tf.executing_eagerly()
tf.random.set_seed(777)

#### dataset 생성

In [14]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

# numpy float format으로 변경
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [15]:
nb_classes = 3

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


#### weight & bias setting

In [16]:
W = tf.Variable(tf.random.normal([4, nb_classes]), name="weight")
b = tf.Variable(tf.random.normal([nb_classes]), name="bias")
variables = [W, b]

print(W, b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.7706481 ,  0.37335405, -0.05576323],
       [ 0.00358377, -0.5898363 ,  1.5702795 ],
       [ 0.2460895 , -0.09918973,  1.4418386 ],
       [ 0.32009882,  0.52678406, -0.7703731 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.3080608 , -0.13253094,  0.5513761 ], dtype=float32)>


#### tf.nn.sofrmax는 softmax activations를 계산해 줌
- softmax = exp(logits) / reduce_sum(exp(logits), dim)

In [17]:
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[1.3657196e-02 7.9016229e-03 9.7844124e-01]
 [3.9259776e-02 1.7034741e-02 9.4370544e-01]
 [3.8038525e-01 1.6772319e-01 4.5189154e-01]
 [3.2339042e-01 5.9075966e-02 6.1753356e-01]
 [3.6299741e-06 6.2072729e-08 9.9999630e-01]
 [2.6252022e-02 1.0727968e-02 9.6301997e-01]
 [1.5652511e-05 4.2180275e-07 9.9998391e-01]
 [2.9407718e-06 3.8113328e-08 9.9999702e-01]], shape=(8, 3), dtype=float32)


#### Softmax onehot test

In [18]:
sample_db = [[8, 2, 1, 4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[0.9302204  0.06200534 0.00777428]], shape=(1, 3), dtype=float32)


In [19]:
def cost_function(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    return cost_mean

print(cost_function(x_data, y_data))

tf.Tensor(6.07932, shape=(), dtype=float32)


In [20]:
# gradient 예시
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x
dy_dx = g.gradient(y, x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [21]:
def gradient_function(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_function(X, Y)
        grads = tape.gradient(loss, variables)
        return grads

print(gradient_function(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.06914605, -0.6509783 ,  0.58183223],
       [-1.5221257 , -1.214863  ,  2.7369888 ],
       [-1.2473829 , -1.7611002 ,  3.008483  ],
       [-1.2014606 , -1.865923  ,  3.0673838 ]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.15212913, -0.342192  ,  0.4943211 ], dtype=float32)>]


In [24]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.1)
    
    for i in range(epochs):
        grads = gradient_function(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if(i == 0) | ((i+1)%verbose == 0):
            print("Loss at epoch {}: {}".format(i+1, cost_function(X, Y).numpy()))

fit(x_data, y_data)

Loss at epoch 1: 2.849416732788086
Loss at epoch 100: 0.6841506958007812
Loss at epoch 200: 0.6138125658035278
Loss at epoch 300: 0.5582041144371033
Loss at epoch 400: 0.5083054900169373
Loss at epoch 500: 0.46105873584747314
Loss at epoch 600: 0.4150715470314026
Loss at epoch 700: 0.3696357309818268
Loss at epoch 800: 0.32453322410583496
Loss at epoch 900: 0.2807207405567169
Loss at epoch 1000: 0.24675193428993225
Loss at epoch 1100: 0.23279790580272675
Loss at epoch 1200: 0.22164484858512878
Loss at epoch 1300: 0.21147581934928894
Loss at epoch 1400: 0.20216412842273712
Loss at epoch 1500: 0.19360587000846863
Loss at epoch 1600: 0.1857142448425293
Loss at epoch 1700: 0.178414985537529
Loss at epoch 1800: 0.17164531350135803
Loss at epoch 1900: 0.1653505563735962
Loss at epoch 2000: 0.15948331356048584


#### Prediction Check

In [25]:
sample_data = [[2, 1, 3, 2]]  # answer_label [[0, 0, 1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1))

tf.Tensor([[0.00112886 0.08154671 0.9173244 ]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [26]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1))

tf.Tensor(
[[2.1975995e-06 1.2331192e-03 9.9876475e-01]
 [1.1288594e-03 8.1546709e-02 9.1732442e-01]
 [2.2205620e-07 1.6418636e-01 8.3581340e-01]
 [6.3921939e-06 8.5045445e-01 1.4953916e-01]
 [2.6150808e-01 7.2644734e-01 1.2044535e-02]
 [1.3783258e-01 8.6213994e-01 2.7417658e-05]
 [7.4242127e-01 2.5754175e-01 3.6978578e-05]
 [9.2197549e-01 7.8023911e-02 6.0005920e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
